In [ ]:
!pip install pandas spacy transformers scikit-learn numpy

In [ ]:
!pip install numpy


In [ ]:
import sys
!{sys.executable} -m pip install numpy


In [ ]:
import shutil

folder_path = r"C:\Users\anvit\myenv\Lib\site-packages\numpy.libs"
shutil.rmtree(folder_path, ignore_errors=True)


In [ ]:
import sys
!{sys.executable} -m pip install --force-reinstall numpy


In [ ]:
import numpy as np
print(np.__version__)


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")


In [ ]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
product_data_input = {
    "name": "Sparkle Clean Multi-Surface Spray",
    "category": "Household Cleaning",
    "description": "A powerful, plant-based cleaning spray that easily removes grease and grime from all surfaces. Leaves a fresh, natural scent.",
    "features": [
        "Plant-based formula",
        "Safe for multiple surfaces (counters, glass, wood)",
        "Streak-free shine",
        "Fresh citrus scent",
        "No harsh chemicals"
    ],
    "benefits": [
        "Makes cleaning quick and easy",
        "Keeps your home safe for kids and pets",
        "Leaves surfaces sparkling clean",
        "Fills your home with a pleasant aroma",
        "Eco-friendly choice"
    ],
    "usps": [ # Unique Selling Points - what makes it special?
        "Dissolves tough grease instantly",
        "Biodegradable ingredients",
        "Comes in a reusable spray bottle"
    ],
    "target_audience": "Busy parents, eco-conscious homeowners",
    "tone_of_voice": "Friendly, effective, trustworthy" # Choose: informative, exciting, professional, etc.
}

In [ ]:
import sys
!{sys.executable} -m pip uninstall -y pillow


In [ ]:
!{sys.executable} -m pip install --no-cache-dir pillow


In [ ]:
from PIL import Image
print("Pillow is working ")


In [ ]:

!pip uninstall -y torch torchvision torchaudio
!pip cache purge


In [4]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available?" if torch.cuda.is_available() else "Using CPU 💻")


Torch version: 2.7.0+cpu
Using CPU 💻


In [26]:
import sklearn
print("Scikit-learn version:", sklearn.__version__)


Scikit-learn version: 1.6.1


In [24]:
import shutil

folder_path = r"C:\Users\anvit\myenv\Lib\site-packages\scipy"
shutil.rmtree(folder_path, ignore_errors=True)


In [27]:
#data preperation
import re
import spacy
from transformers import pipeline, set_seed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the English language model for text understanding
nlp = spacy.load("en_core_web_sm")

# Set up the text generation AI model (Flan-T5 is a good general-purpose model)
text_generator = pipeline('text2text-generation', model="google/flan-t5-base", device=-1) # -1 means use CPU
set_seed(42) # helps get similar results each time you run it

# --- Helper Functions ---
def clean_text(text):
    text = str(text) 
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

def create_prompt(data, length="medium", tone="informative", audience="general consumers"):
    # instruction for the AI
    product_name = data.get("name", "A fantastic product")
    category = data.get("category", "general merchandise")
    features = ", ".join(data.get("features", []))
    benefits = ", ".join(data.get("benefits", []))
    usps = ", ".join(data.get("usps", []))

    length_instructions = {
        "short": "a concise, impactful 2-3 sentence description",
        "medium": "a detailed 4-6 sentence description, covering key aspects",
        "long": "a comprehensive 8-12 sentence description, highlighting all features and benefits"
    }.get(length, "a detailed description")

    prompt = f"""Generate {length_instructions} for a product.
Product Name: {product_name}
Category: {category}
Key Features: {features}
Benefits: {benefits}
Unique Selling Points: {usps}
Target Audience: {audience}
Tone of Voice: {tone}

Make sure the copy is engaging, highlights benefits, and encourages purchase.
"""
    return prompt

#process input product data
processed_product_info = {
    "name": clean_text(product_data_input.get("name", "")),
    "category": clean_text(product_data_input.get("category", "")),
    "raw_description": clean_text(product_data_input.get("description", "")),
    "features": [clean_text(f) for f in product_data_input.get("features", []) if f],
    "benefits": [clean_text(b) for b in product_data_input.get("benefits", []) if b],
    "usps": [clean_text(u) for u in product_data_input.get("usps", []) if u],
    "target_audience": clean_text(product_data_input.get("target_audience", "general consumers")),
    "tone_of_voice": clean_text(product_data_input.get("tone_of_voice", "informative"))
}

if not processed_product_info["benefits"] and processed_product_info["features"]:
    processed_product_info["benefits"] = processed_product_info["features"]

# extract initial keywords from basic info
initial_keywords = [
    processed_product_info["name"].lower(),
    processed_product_info["category"].lower()
]
initial_keywords.extend([f.lower() for f in processed_product_info["features"]])
initial_keywords.append(f"{processed_product_info['category'].lower()} {processed_product_info['name'].lower()}")

print("--- Product Data Processed ---")
print(f"Product Name: {processed_product_info['name']}")
print(f"Category: {processed_product_info['category']}")
print(f"Initial Keywords: {initial_keywords}\n")

Device set to use cpu


--- Product Data Processed ---
Product Name: Sparkle Clean Multi-Surface Spray
Category: Household Cleaning
Initial Keywords: ['sparkle clean multi-surface spray', 'household cleaning', 'plant-based formula', 'safe for multiple surfaces (counters, glass, wood)', 'streak-free shine', 'fresh citrus scent', 'no harsh chemicals', 'household cleaning sparkle clean multi-surface spray']



In [39]:
#product copy

def generate_product_copy(data, length="medium", num_variations=2):
    prompts = []
    
    # Define the instruction based on length
    length_prompt_map = {
        "short": "Write a catchy and concise product description (1-2 lines).",
        "medium": "Write a detailed product description (1 paragraph) that highlights the features and benefits.",
        "long": "Write a long, persuasive product description (3 paragraphs) that emotionally connects with the target audience and explains how the product improves their life."
    }

    for _ in range(num_variations):
        prompt = f"""{length_prompt_map.get(length, '')}

Product Name: {data.get('product_name')}
Product Category: {data.get('product_category')}
Product Features: {', '.join(data.get('features', []))}
Target Audience: {data.get('target_audience')}
Tone: {data.get('tone_of_voice')}
"""
        prompts.append(prompt)

    generated_texts = []
    for p in prompts:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # or "gpt-4" if you have access
            messages=[
                {"role": "system", "content": "You are a professional copywriter."},
                {"role": "user", "content": p}
            ],
            temperature=0.7,
            top_p=0.9,
            max_tokens=300,
            n=1
        )
        copy_text = response.choices[0].message["content"].strip()
        generated_texts.append(copy_text)

    return generated_texts


print("Generating Product Copy (Short)")
short_copies = generate_product_copy(processed_product_info, length="short", num_variations=2)
for i, copy in enumerate(short_copies):
    print(f"Short Copy {i+1}:\n{copy}\n")

print("Generating Product Copy (Medium)")
medium_copies = generate_product_copy(processed_product_info, length="medium", num_variations=2)
for i, copy in enumerate(medium_copies):
    print(f"Medium Copy {i+1}:\n{copy}\n")

print("Generating Product Copy (Long)")
long_copies = generate_product_copy(processed_product_info, length="long", num_variations=1)
for i, copy in enumerate(long_copies):
    print(f"Long Copy {i+1}:\n{copy}\n")

# combine all generated copies for keyword suggestion
all_generated_copies = short_copies + medium_copies + long_copies

Generating Product Copy (Short)
Short Copy 1:
Sparkle Clean Multi-Surface Spray is a reusable spray bottle that is safe for multiple surfaces. It is a plant-based formula that is safe for multiple surfaces (counters, glass, wood). It has a citrus

Short Copy 2:
Sparkle Clean Multi-Surface Spray is a household cleaning spray for all surfaces. It's non-toxic and leaves surfaces sparkling clean. It's also biodegradable.

Generating Product Copy (Medium)
Medium Copy 1:
Sparkle Clean Multi-Surface Spray is a multi-purpose cleaning spray that is designed to clean multiple surfaces. It is available in a reusable spray bottle and is formulated to remove tough grease.

Medium Copy 2:
Sparkle Clean Multi-Surface Spray is a house cleaning spray that is made with a plant-based formula that can be used to clean multiple surfaces. It is safe for multiple surfaces, and is non-toxic. The spray can be used on all types of surfaces including glass, counters, glass, wood, and counters.

Generating Produc

In [40]:
#generate keywords

def extract_keywords(product_data, generated_copies, initial_kws, num_suggestions=10):
    stop_words = spacy.lang.en.stop_words.STOP_WORDS

    # combine all relevant text into one big string
    all_combined_text = " ".join([product_data["name"], product_data["category"],
                                  product_data["raw_description"]] +
                                 product_data["features"] +
                                 product_data["benefits"] +
                                 product_data["usps"] +
                                 generated_copies)

    # use spaCy to find important noun phrases 
    doc = nlp(all_combined_text.lower())
    phrases = [chunk.text for chunk in doc.noun_chunks]

    # combine initial keywords with new phrases
    potential_keywords = list(set(initial_kws + phrases))
    # Filter out very short or stop-word heavy phrases
    potential_keywords = [kw for kw in potential_keywords if len(kw.split()) <= 4 and kw not in stop_words]

    if not potential_keywords:
        return [] # no keywords to suggest

    # use TF-IDF to find important words in the text
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
    try:
        
        tfidf_matrix = vectorizer.fit_transform([all_combined_text] + potential_keywords)
    except ValueError:
        return list(set(initial_kws)) # Fallback if TF-IDF fails

    # calculate how similar each potential keyword is to the main text
    text_vector = tfidf_matrix[0:1]
    keyword_vectors = tfidf_matrix[1:]
    similarities = cosine_similarity(text_vector, keyword_vectors).flatten()

    # sort keywords by their similarity score
    keyword_scores = sorted(zip(potential_keywords, similarities), key=lambda x: x[1], reverse=True)

    # for top relevant keywords
    suggested = [kw for kw, score in keyword_scores if score > 0.01] # Filter out very low scores

    final_keywords = []
    seen = set()
    for kw in suggested:
        if kw not in seen:
            final_keywords.append(kw)
            seen.add(kw)
        if len(final_keywords) >= num_suggestions:
            break

    # ensure product name and category are usually among the top suggestions
    if product_data["name"].lower() not in final_keywords[:num_suggestions]:
        final_keywords.insert(0, product_data["name"].lower())
    if product_data["category"].lower() not in final_keywords[:num_suggestions]:
        final_keywords.insert(1, product_data["category"].lower())


    return final_keywords[:num_suggestions]


print(" Suggesting Keywords ")
suggested_keywords = extract_keywords(
    processed_product_info,
    all_generated_copies,
    initial_keywords,
    num_suggestions=15
)
print(f"Suggested Keywords: {suggested_keywords}\n")

 Suggesting Keywords 
Suggested Keywords: ['sparkle clean multi-surface spray', 'multiple surfaces', 'the spray', 'a household cleaning spray', 'a reusable spray bottle', 'surfaces', 'all surfaces', 'a multi-purpose cleaning spray', 'clean', 'a plant-based formula', 'plant-based formula', 'a house cleaning spray', 'household cleaning', 'clean fills', 'cleaning']



In [42]:

print("\n Product Example: Smartwatch")
smartwatch_data_input = {
    "name": "FuturaX Smartwatch Pro",
    "category": "Wearable Technology",
    "description": "Advanced smartwatch with health monitoring, GPS, and long battery life. Stay connected and track your fitness goals effortlessly.",
    "features": [
        "Heart rate monitoring",
        "Blood oxygen sensor",
        "Built-in GPS",
        "5-day battery life",
        "Water resistant (5 ATM)",
        "AMOLED display",
        "Smartphone notifications"
    ],
    "benefits": [
        "Comprehensive health tracking",
        "Accurate outdoor activity tracking",
        "Freedom from daily charging",
        "Suitable for swimming",
        "Vibrant and clear visuals",
        "Never miss an important alert"
    ],
    "usps": [
        "Industry-leading battery life for a smartwatch of its class",
        "Seamless integration with popular fitness apps",
        "Sleek and durable design"
    ],
    "target_audience": "Fitness enthusiasts, tech-savvy individuals, busy professionals",
    "tone_of_voice": "Modern, authoritative, exciting"
}

# Process the new product data
processed_smartwatch_info = {
    "name": clean_text(smartwatch_data_input.get("name", "")),
    "category": clean_text(smartwatch_data_input.get("category", "")),
    "raw_description": clean_text(smartwatch_data_input.get("description", "")),
    "features": [clean_text(f) for f in smartwatch_data_input.get("features", []) if f],
    "benefits": [clean_text(b) for b in smartwatch_data_input.get("benefits", []) if b],
    "usps": [clean_text(u) for u in smartwatch_data_input.get("usps", []) if u],
    "target_audience": clean_text(smartwatch_data_input.get("target_audience", "general consumers")),
    "tone_of_voice": clean_text(smartwatch_data_input.get("tone_of_voice", "informative"))
}
if not processed_smartwatch_info["benefits"] and processed_smartwatch_info["features"]:
    processed_smartwatch_info["benefits"] = processed_smartwatch_info["features"]

initial_smartwatch_keywords = [
    processed_smartwatch_info["name"].lower(),
    processed_smartwatch_info["category"].lower()
]
initial_smartwatch_keywords.extend([f.lower() for f in processed_smartwatch_info["features"]])
initial_smartwatch_keywords.append(f"{processed_smartwatch_info['category'].lower()} {processed_smartwatch_info['name'].lower()}")

print("\n Processed Smartwatch Data ")
print(f"Product Name: {processed_smartwatch_info['name']}")
print(f"Category: {processed_smartwatch_info['category']}")
print(f"Initial Keywords: {initial_smartwatch_keywords}\n")

# Generate copy for the smartwatch
print(" Generating Smartwatch Copy (Medium) ")
smartwatch_medium_copies = generate_product_copy(processed_smartwatch_info, length="medium", num_variations=1)
for i, copy in enumerate(smartwatch_medium_copies):
    print(f"Smartwatch Medium Copy {i+1}:\n{copy}\n")

# Suggest keywords for the smartwatch
print(" Suggesting Smartwatch Keywords ")
suggested_smartwatch_keywords = extract_keywords(
    processed_smartwatch_info,
    smartwatch_medium_copies, # Using the generated copy for keywords
    initial_smartwatch_keywords,
    num_suggestions=15
)
print(f"Suggested Smartwatch Keywords: {suggested_smartwatch_keywords}\n")


 Product Example: Smartwatch

 Processed Smartwatch Data 
Product Name: FuturaX Smartwatch Pro
Category: Wearable Technology
Initial Keywords: ['futurax smartwatch pro', 'wearable technology', 'heart rate monitoring', 'blood oxygen sensor', 'built-in gps', '5-day battery life', 'water resistant (5 atm)', 'amoled display', 'smartphone notifications', 'wearable technology futurax smartwatch pro']

 Generating Smartwatch Copy (Medium) 
Smartwatch Medium Copy 1:
The FuturaX Smartwatch Pro is a smartwatch that monitors your heart rate, blood oxygen, and blood pressure. It is designed to be worn in the water and is waterproof. Its battery life is 5 days.

 Suggesting Smartwatch Keywords 
Suggested Smartwatch Keywords: ['its battery life', 'wearable technology', 'futurax smartwatch pro', '5-day battery life', 'long battery life', 'the futurax smartwatch', 'a smartwatch', 'blood oxygen sensor', 'heart rate monitoring', 'blood oxygen', 'heart rate', 'your heart rate', 'health monitoring', 'pop